# Kalman filter

In [1]:
import sys
sys.path.append(r"C:\Users\ybr5070\yolov7")  # Update with your actual path

import cv2
import torch
import torchvision.transforms.functional as F
import os
import numpy as np
import winsound

from models.experimental import attempt_load  # For YOLOv7 model loading
from utils.general import non_max_suppression  # For YOLOv7 post-processing
from collections import deque
from pykalman import KalmanFilter

def get_frame_timestamps(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_timestamps = {i: i / fps for i in range(frame_count)}
    cap.release()
    return frame_timestamps

def clean_path(path):
    return "\\".join([part.strip() for part in path.split("\\")])

save_base_dir = clean_path(r"C:\Users\ybr5070\Desktop\yolo_face\PS4")

def save_tracked_image(frame, bbox, output_dir, timestamp):
    try:
        if bbox is not None:
            x, y, w, h = [int(v) for v in bbox]
            roi_frame = frame[y:y+h, x:x+w]
            if roi_frame.size > 0:
                roi_frame_resized = cv2.resize(roi_frame, (224, 224))  # Resize to 224x224
                filename = f"tracked_{timestamp:.2f}.jpg"
                filepath = clean_path(os.path.join(output_dir, filename))
                success = cv2.imwrite(filepath, roi_frame_resized)
                if not success:
                    raise Exception("cv2.imwrite returned False")
    except Exception as e:
        print(f"Exception occurred while saving image: {e}")

def process_frame(frame, target_size=(512, 320)):
    frame_resized = cv2.resize(frame, target_size)
    tensor_frame = F.to_tensor(frame_resized).unsqueeze(0).cuda()
    return tensor_frame, frame_resized

def initialize_kalman_filter():
    kf = KalmanFilter(
        transition_matrices=[[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]],
        observation_matrices=[[1, 0, 0, 0], [0, 1, 0, 0]],
    )
    return kf

def update_kalman(kf, state_mean, state_covariance, observation):
    state_mean, state_covariance = kf.filter_update(
        filtered_state_mean=state_mean,
        filtered_state_covariance=state_covariance,
        observation=observation
    )
    return state_mean, state_covariance

def detect_and_track(video_path, model, output_dir, timestamps, redetect_interval=10, target_size=(512, 320), detection_threshold=0.2, nms_threshold=0.8, confidence_boost_threshold=5):
    print("detect_and_track called")
    cap = cv2.VideoCapture(video_path)
    tracker = cv2.TrackerKCF_create()
    kf = initialize_kalman_filter()  # Initialize Kalman Filter
    init_tracking = False
    frame_count = 0
    label = ""
    screen_width, screen_height = 1366, 768  # Set display size
    tracked_confidence = 0
    tracking_stability = 0
    track_history = deque(maxlen=10)  # History for trajectory prediction
    state_mean = None
    state_covariance = np.eye(4)
    padding = 5

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = timestamps[frame_count]

        if frame_count % redetect_interval == 0 or not init_tracking:
            tensor_frame, resized_frame = process_frame(frame, target_size)
            model.eval()
            with torch.no_grad():
                prediction = model(tensor_frame)
                prediction = prediction[0] if isinstance(prediction, tuple) else prediction
                prediction = non_max_suppression(prediction, detection_threshold, nms_threshold)

            if len(prediction[0]) > 0:
                element = prediction[0][0]  # Take the first detection only
                tracked_bbox = element[:4].cpu().numpy()
                x, y, x2, y2 = tracked_bbox
                # Scaling the bounding box coordinates to the original frame
                scale_x = frame.shape[1] / target_size[0]
                scale_y = frame.shape[0] / target_size[1]
                x = int(x * scale_x) - padding
                y = int(y * scale_y) - padding
                x2 = int(x2 * scale_x) + padding
                y2 = int(y2 * scale_y) + padding
                # Ensure bbox is within frame boundaries
                x = max(0, x)
                y = max(0, y)
                x2 = min(frame.shape[1], x2)
                y2 = min(frame.shape[0], y2)
                w, h = x2 - x, y2 - y
                conf = element[4].cpu().numpy()
                tracker = cv2.TrackerKCF_create()
                tracker.init(frame, (x, y, w, h))
                init_tracking = True
                tracked_confidence = conf
                tracking_stability = 1
                track_history.append((x + w / 2, y + h / 2))  # Center of bbox
                state_mean = np.array([x + w / 2, y + h / 2, 0, 0])  # Update state_mean
                # Draw bounding box with confidence and label
                label = f"child_face {conf:.2f}"
                display_x, display_y = int(x / scale_x), int(y / scale_y)
                display_w, display_h = int(w / scale_x), int(h / scale_y)
                cv2.rectangle(resized_frame, (display_x, display_y), (display_x + display_w, display_y + display_h), (255, 0, 0), 1)
                cv2.putText(resized_frame, label, (display_x, display_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
                save_tracked_image(frame, (x, y, w, h), output_dir, timestamp)
            else:
                init_tracking = False
        else:
            tensor_frame, resized_frame = process_frame(frame, target_size)

        if init_tracking:
            success, tracked_bbox = tracker.update(frame)
            if success:
                tracking_stability += 1
                if tracking_stability >= confidence_boost_threshold:
                    tracked_confidence = min(tracked_confidence + 0.1, 1.0)  # Boost confidence
                x, y, w, h = [int(v) for v in tracked_bbox]
                x = max(0, x - padding)
                y = max(0, y - padding)
                x2 = min(frame.shape[1], x + w + 2 * padding)
                y2 = min(frame.shape[0], y + h + 2 * padding)
                w, h = x2 - x, y2 - y
                track_history.append((x + w / 2, y + h / 2))  # Center of bbox
                state_mean = np.array([x + w / 2, y + h / 2, 0, 0])  # Update state_mean
                # Convert the bbox back to the resized frame scale
                display_x, display_y = int(x / scale_x), int(y / scale_y)
                display_w, display_h = int(w / scale_x), int(h / scale_y)
                # Draw bounding box with confidence and label
                cv2.rectangle(resized_frame, (display_x, display_y), (display_x + display_w, display_y + display_h), (255, 0, 0), 1)
                cv2.putText(resized_frame, f"child_face {tracked_confidence:.2f}", (display_x, display_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
                save_tracked_image(frame, (x, y, w, h), output_dir, timestamp)
            else:
                # Use Kalman Filter to predict the next position
                if len(track_history) >= 2 and state_mean is not None:
                    predicted_pos, state_covariance = update_kalman(kf, state_mean, state_covariance, track_history[-1])
                    x, y = int(predicted_pos[0] - w / 2), int(predicted_pos[1] - h / 2)
                    track_history.append((predicted_pos[0], predicted_pos[1]))
                    cv2.rectangle(resized_frame, (x, y), (x + w, y + h), (0, 255, 0), 1)  # Draw predicted bbox
                    cv2.putText(resized_frame, "Predicted", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
                    save_tracked_image(frame, (x, y, w, h), output_dir, timestamp)

                tracking_stability = 0
                tracked_confidence = max(tracked_confidence - 0.1, 0)  # Decrease confidence if tracking fails

        # Resize frame for display purposes
        display_frame = cv2.resize(resized_frame, (screen_width, screen_height))
        cv2.imshow("Frame", display_frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

        frame_count += 1

    if frame_count > 0:
        last_timestamp = timestamps[frame_count - 1]
        print(f"Last timestamp processed for frame {frame_count}: {last_timestamp}")
    else:
        print("No frames were processed.")

    cap.release()
    cv2.destroyAllWindows()


def process_videos(directory_path, model_path, save_base_dir):
    if not os.path.exists(directory_path):
        print(f"Directory path {directory_path} does not exist.")
        return
    if not os.path.exists(model_path):
        print(f"Model path {model_path} does not exist.")
        return
    
    model = attempt_load(model_path, map_location='cuda')  # Load YOLOv7 model
    for filename in os.listdir(directory_path):
        if filename.lower().endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(directory_path, filename)
            print(f"Processing video: {filename}")

            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Failed to open video {filename}")
                continue

            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            if frame_count <= 0 or fps <= 0:
                print(f"Invalid video properties for {filename}")
                cap.release()
                continue

            print(f"Frame Count (CAP_PROP_FRAME_COUNT): {frame_count}")
            timestamps = get_frame_timestamps(video_path)

            ret, first_frame = cap.read()
            if not ret:
                print(f"Failed to read first frame of {filename}")
                cap.release()
                continue
            cap.release()
            winsound.Beep(440, 500)

            video_save_dir = os.path.join(save_base_dir, os.path.splitext(filename)[0])
            os.makedirs(video_save_dir, exist_ok=True)

            detect_and_track(video_path, model, video_save_dir, timestamps)
            print(f"Finished processing {filename}.")

# Example usage
process_videos(r"C:\Users\ybr5070\Documents\PS4_fixed", r"C:\Users\ybr5070\yolov7\runs\train\exp4\weights\best.pt", save_base_dir)


Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


c:\Users\ybr5070\AppData\Local\anaconda3\envs\yolo_env\Lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Processing video: R01_090_V4_PS4_fixed.mp4
Frame Count (CAP_PROP_FRAME_COUNT): 21345
detect_and_track called
Last timestamp processed for frame 21345: 711.4666666666667
Finished processing R01_090_V4_PS4_fixed.mp4.
Processing video: R01_093_V4_PS4_fixed.mp4
Frame Count (CAP_PROP_FRAME_COUNT): 21394
detect_and_track called
Last timestamp processed for frame 21394: 713.1
Finished processing R01_093_V4_PS4_fixed.mp4.
Processing video: R01_095_V3_PS4_fixed.mp4
Frame Count (CAP_PROP_FRAME_COUNT): 35633
detect_and_track called
Last timestamp processed for frame 35633: 1187.7333333333333
Finished processing R01_095_V3_PS4_fixed.mp4.
Processing video: R01_096_V2_PS4_fixed.mp4
Frame Count (CAP_PROP_FRAME_COUNT): 71100
detect_and_track called
Last timestamp processed for frame 71100: 2369.9666666666667
Finished processing R01_096_V2_PS4_fixed.mp4.
Processing video: R01_098_V3_PS4_fixed.mp4
Frame Count (CAP_PROP_FRAME_COUNT): 11916
detect_and_track called
Last timestamp processed for frame 11916: